In [1]:
import json
from torch.utils.data import Dataset, DataLoader

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f64360684c0>>
Traceback (most recent call last):
  File "/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
def load_finqa_dataset(path):
    with open(path, "r") as f:
        return json.load(f)
    
train_data_finqa = load_finqa_dataset("/cs/student/projects1/aibh/2024/tpatil/comp0087/FrugalML/datasets/FinQA/train.json")
val_data_finqa = load_finqa_dataset("/cs/student/projects1/aibh/2024/tpatil/comp0087/FrugalML/datasets/FinQA/dev.json")
test_data_finqa = load_finqa_dataset("/cs/student/projects1/aibh/2024/tpatil/comp0087/FrugalML/datasets/FinQA/test.json")

In [23]:
examples = train_dataset_finqa[0]
examples

{'pre_text': ['interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .',
  'if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million .',
  'foreign currency exposure as more fully described in note 2i .',
  'in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s .',
  'dollar-based exposures by entering into forward foreign currency exchange contracts .',
  'the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months .',
  'currently , our largest foreign currency exposure is the euro , primarily because our european operations have the highest proportion of our local currency denominated expenses .',
  'relative to foreign currency exposures existing at october 31 , 2009 and november 1 , 2008 , 

In [24]:
examples['table']

[['', 'october 31 2009', 'november 1 2008'],
 ['fair value of forward exchange contracts asset ( liability )',
  '$ 6427',
  '$ -23158 ( 23158 )'],
 ['fair value of forward exchange contracts after a 10% ( 10 % ) unfavorable movement in foreign currency exchange rates asset ( liability )',
  '$ 20132',
  '$ -9457 ( 9457 )'],
 ['fair value of forward exchange contracts after a 10% ( 10 % ) favorable movement in foreign currency exchange rates liability',
  '$ -6781 ( 6781 )',
  '$ -38294 ( 38294 )']]

In [25]:
table = examples['table']
table_prompt=""
table_prompt += f"\nTable:\n"
for row in table:
    table_prompt += "|".join([str(cell) for cell in row]) + " \n"
table_prompt

'\nTable:\n|october 31 2009|november 1 2008 \nfair value of forward exchange contracts asset ( liability )|$ 6427|$ -23158 ( 23158 ) \nfair value of forward exchange contracts after a 10% ( 10 % ) unfavorable movement in foreign currency exchange rates asset ( liability )|$ 20132|$ -9457 ( 9457 ) \nfair value of forward exchange contracts after a 10% ( 10 % ) favorable movement in foreign currency exchange rates liability|$ -6781 ( 6781 )|$ -38294 ( 38294 ) \n'

In [44]:
def create_prompt_instance(table, pre_text, post_text, question, answer, derivation=None):
    set_context = "You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\n"
    table_prompt = "The table provide the financial data. All the elements in the table are separated by \"|\". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.\n"
    # paragraph_prompt = "The paragraphs provides some context about the data in the table. It may contain information that is not present in the table. It may also contain some numbers which might require arithmatic processing to get the answer. There may be multiple paragraphs separated by keyword matching \"Paragraph [0-9]+:\". Interpret each paragraph and use the data and description in it to infer the answer to the provided quesions.\n"
    pre_text_prompt = "The pre_text provides some context about the data before the table. It may contain information that is not present in the table. It may also contain some numbers which might require arithmatic processing to get the answer. There may be multiple sentences separated by comma (,). Interpret each pre_text paragraph and use the data and description in it to infer the answer to the provided quesions.\n"
    post_text_prompt = "The post_text provides some context about the data after the table. It may contain information that is not present in the table. It may also contain some numbers which might require arithmatic processing to get the answer. There may be multiple sentences separated by comma (,). Interpret each post_text paragraph and use the data and description in it to infer the answer to the provided quesions.\n"
    question_prompt = "The question is asked based on the data in the table and the paragraph. You are expected to answer the question based on the data in the table and the paragraph.\n"
    answer_prompt = "" #"You are expected to answer the question based on the data in the table and the paragraph. Provide only the answer to the question and do not repeat the question. Use the answers provided as labels to learn the correct way to answer the question.\n"
    # derivation_prompt = "The derivation provides the steps to calculate the answer to the question. You can learn how to use the derivation to calculate the answer to the question.\n"
    answer_instruction_prompt = "\nInstruction: Answer the question based on the data in the table and the paragraph. Provide only the answer to the question and do not repeat the question. Use the answers provided as labels to learn the correct way to answer the question.\n"

    table_prompt += f"\nTable:\n"
    for row in table:
        table_prompt += "|".join([str(cell) for cell in row]) + " \n"

    pre_text_prompt += f"\nPre_text:\n {pre_text}"
    post_text_prompt += f"\nPost_text:\n {post_text}"
    question_prompt += f"\nQuestion:\n {question}"
    answer_prompt += f"\nAnswer:\n {answer}"
    # derivation_prompt += f"\nDerivation:\n {derivation}"

    return set_context, table_prompt, pre_text_prompt, post_text_prompt, question_prompt, answer_prompt, answer_instruction_prompt #, derivation_prompt

In [45]:
class FinQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.qa_pairs = []

        for item in self.data:
        # item = self.data[idx]
            tables = item.get('table', [])
            pre_text = item.get('pre_text', [])
            post_text = item.get('post_text', [])
            qa = item.get('qa', {})

            question = qa["question"].strip()
            answer = qa["answer"]
            # derivation = question_answer.get("derivation", "")
            # answer_type = question_answer.get("answer_type", "")
            # answer_from = question_answer.get("answer_from", "")
            # facts = question_answer.get("facts", "")
            # answer_mapping = question_answer.get("mapping", "")
            # scale = question_answer.get("scale", "")
            # rel_paragraphs = question_answer.get("rel_paragraphs", "")
            # req_comparison = question_answer.get("req_comparison", "")

            set_context, table_prompt, pre_text_prompt, post_text_prompt, question_prompt, answer_prompt, answer_instruction_prompt = create_prompt_instance(tables, pre_text, post_text, question, answer)
            input_context = (set_context + pre_text_prompt + table_prompt + post_text_prompt + question_prompt + answer_instruction_prompt).strip()
            # label_text = (answer_prompt + derivation_prompt + answer_type_prompt + answer_from_prompt + facts_prompt + answer_mapping_prompt + scale_prompt).strip()
            label_text = answer_prompt.strip()

            # qa_pairs.append({
            #     "input_ids": inputs.input_ids.squeeze(),
            #     "attention_mask": inputs.attention_mask.squeeze(),
            #     "labels": labels
            # })

            self.qa_pairs.append((
                input_context,
                label_text
            ))
        # print(self.qa_pairs[-1])

    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_context, label_text = self.qa_pairs[idx]

        # inputs = self.tokenizer( 
        #             input_context, 
        #             # max_length=self.max_length, 
        #             # truncation=True, 
        #             # padding="max_length", 
        #             return_tensors="pt"
        #         )

        # labels = self.tokenizer(
        #             label_text, 
        #             # max_length=self.max_length, 
        #             # truncation=True, 
        #             # padding="max_length", 
        #             return_tensors="pt"
        #         )

        # input_ids = inputs["input_ids"].squeeze(0)
        # attention_mask = inputs["attention_mask"].squeeze(0)
        # labels = labels["input_ids"].squeeze(0)


        return {
            # "input_ids": input_ids,
            # "attention_mask": attention_mask,
            # "labels": labels
            "input_context": input_context,
            "label_text": label_text
        }

In [46]:
from transformers import AutoTokenizer
model_name = "microsoft/Phi-3.5-mini-instruct"
cache_dir = "/cs/student/projects1/aibh/2024/tpatil/.cache/huggingface"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)

In [47]:
train_dataset_finqa = FinQADataset(train_data_finqa, tokenizer)

train_dataloader_finqa = DataLoader(
    train_dataset_finqa, 
    batch_size=1, 
    shuffle=False
)

In [48]:
next(iter(train_dataloader_finqa))

{'input_context': ['You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\nThe pre_text provides some context about the data before the table. It may contain information that is not present in the table. It may also contain some numbers which might require arithmatic processing to get the answer. There may be multiple sentences separated by comma (,). Interpret each pre_text paragraph and use the data and description in it to infer the answer to the provided quesions.\n\nPre_text:\n [\'interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .\', \'if libor changes by 100 basis points , our annual interest expense would change by